In [ ]:
import numpy as np
import pandas as pd
##### WEB scriping #####
import requests
from bs4 import BeautifulSoup
########################
from datetime import datetime, date, timedelta
from googletrans import Translator 

<font style="font-size: 80px"><font color="yellow">C</font>onsume <font color="yellow">P</font>roduct <font color="yellow">I</font>ndex (<font color="yellow">CPI</font>)</font>

In [ ]:
CPIworld = pd.read_csv(r"C:\Users\matia\OneDrive\Documentos\Proyectos\Data proyect\CSV crudo\CPIworld.csv")
CPIworld

### Columns title

In [ ]:
CPIworld.drop(columns=["Indicator Name", "Indicator Code", "Unnamed: 66"], inplace=True)
CPIworld.rename(columns={"Country Name":"Country", "Country Code":"Code"}, inplace=True)
CPIworld

In [ ]:
country_check = {}
for i in CPIworld["Country"]:
    if country_check.get(i):
        country_check[i] += 1
    else:
        country_check[i] = 1
        
country_repeat = {k: v for k, v in country_check.items() if(v>=2)}
country_repeat

In [ ]:
CPIworld.info()

### <font style="font-style:italic "><font color="#6CACE4">This data-set</font> <font color="white">not have Argentina</font> <font color="yellow">data.That</font> <font color="white">is my country</font> <font color="#6CACE4">where I born.</font></font> <font style="font-size:15px">(WEB scriping)</font>

In [ ]:
print(CPIworld[CPIworld["Country"] == "Argentina"].iloc[:, 2:].isna().any().value_counts(normalize=True)) #False = 0 => All is nan
CPIworld.drop(index=CPIworld[CPIworld["Country"] == "Argentina" ].index.values, inplace=True)

##### WEB scriping

In [ ]:
url = "https://www.datosmundial.com/america/argentina/inflacion.php"
page = requests.get(url)
soup = BeautifulSoup(page.text, 'html.parser')

tr_items = soup.find_all('td')

In [ ]:
Argentina_inflation = {}
count_year = 0
count_Argentina = 1

for td in tr_items:
    if(count_year == 0):
        actual_year = td.text
        count_year = 5

    elif(count_Argentina == 0):
        Argentina_inflation[actual_year] = td.text
        count_Argentina = 5

    count_year -= 1
    count_Argentina -= 1

Argentina_inflation

In [ ]:
dict_aux = {}
for key, value in Argentina_inflation.items():
    dict_aux[key] = float(value.split("%")[0].replace(".", "").replace(",", "."))   #How to change "2.313,98"

del dict_aux["2022"]
dict_aux

In [ ]:
addRow = pd.DataFrame([dict_aux])
addRow = addRow.assign(Country="Argentina", Code="ARG")
print(CPIworld.info())
print(addRow.info())

df_test =pd.concat([CPIworld, addRow], ignore_index=True)
#******
false_data = df_test[df_test["Country"] == "Argentina"].iloc[:, 22:].isna().any()
print("No data years: ", false_data.index[false_data].values)
#******
CPIworld = df_test

In [ ]:
CPIworld

In [ ]:
int(CPIworld[CPIworld["Country"] == "Argentina"].index.values)

### NaN content

In [ ]:
dataDelete = CPIworld[CPIworld["Country"].isna()==True]
print(dataDelete.iloc[:, :2], "\n", dataDelete.index)
CPIworld.drop(index=dataDelete.index, inplace=True) #index=[1, 3]
CPIworld.reset_index(inplace=True, drop=True)

In [ ]:
noData = []
for i in CPIworld["Country"]:
    rowSelected = CPIworld[CPIworld["Country"] == i]
    
    if(rowSelected.iloc[:, 2:].isna().any().any()):
        TrueCount = rowSelected.iloc[:, 2:].isna().any().value_counts(normalize=True)[True]
        print(rowSelected.index.values, i, "have a", (TrueCount*100).round(2), "%", "NaN values")
        if(TrueCount == 1):
            noData.append(int(rowSelected.index.values))
        else:
            pass
    else:
        pass
print(len(noData), noData)

CPIworld.drop(index=noData, inplace=True)
CPIworld.reset_index(inplace=True, drop=True)
CPIworld

In [ ]:
nanCountry = {}

for i in CPIworld["Country"]:
    nanCountry[i] = CPIworld[CPIworld["Country"] == i].iloc[:, 2:].isna().any().value_counts(normalize=True)[False]
    nanCountry[i] *= 100

nanCountryOrder = dict(sorted(nanCountry.items(), key=lambda items: items[1], reverse=True))
nanCountryOrder

### Order information

In [ ]:
auxDataframe = pd.DataFrame(columns=CPIworld.columns)
print(auxDataframe)


for i, u in zip(nanCountryOrder, range(len(nanCountryOrder))):
    rowAdd = {x:np.nan for x in CPIworld.columns}
    rowSelected = CPIworld[CPIworld["Country"] == i]

    for w in auxDataframe.columns:
        rowAdd[w] = rowSelected[w].values[0]
    
    new_df = pd.DataFrame([rowAdd])
    auxDataframe = pd.concat([auxDataframe, new_df], ignore_index=True)
    
CPIworld = auxDataframe
CPIworld = CPIworld.round(2)
CPIworld

### <font color="orange">Countrys that need more data</font>

In [ ]:
CPIworld_extra = pd.DataFrame()

# <font color="green">Save changes</font>

In [ ]:
CPIworld.to_csv(r"C:\Users\matia\OneDrive\Documentos\Proyectos\Data proyect\Clean\IPCyear\CPIworld.csv")

In [ ]:
CPIworld[CPIworld["Country"] == "Mundo"]

In [ ]:
CPIworld